## Abacat tutorial


### Getting started
Using Abacat is quite straightforward, and you will find that it can be used inline (like in this tutorial) and also in the command line.

Abacat's main class is the `Genome` class, which will hold most of your data.

In [1]:
# Let's instanstiate a genome to start.
from abacat import Genome

g = Genome()

### Loading data
Well, our class doesn't hold anything for now, but we can load a .fasta file containing WGS data. Abacat comes with 7 genomes so you can play around without worrying with downloading data.

Our genomes are located in `abacat/data/genomes`.

For now, let's load a genome unto our genome instance. This is a cyanobacterial genome of the species **Synechococcus elongatus**, but it comes named for its NCBI accession number. More information about it is available [here](https://www.ncbi.nlm.nih.gov/assembly/GCF_000012525.1/), at the NCBI Assembly database.

In [2]:
g.load_contigs("abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.fna")

2019-09-27 06:10:08 - Contigs file set as /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.fna
2019-09-27 06:10:08 - Directory set as /Users/viniWS/Bio/abacat/abacat/data/genomes
2019-09-27 06:10:08 - Name set as GCF_000012525.1_ASM1252v1_genomic


Notice that Abacat sets `directory` and `name` attributes. Our file is stored in `g.files`.

In [3]:
g.files

{'contigs': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.fna'}

`g.files` is a dictionary because we will generate more files and their paths will be stored there. We can have a look at our sequence statistics using [seqstats](https://github.com/clwgg/seqstats), a quick command line tool which Abacat provides a wrapper for:

In [4]:
g.load_seqstats()
g.seqstats  # We also store this information as an attribute.

{'Total n': 2.0,
 'Total seq': 2742269.0,
 'Avg. seq': 1371134.5,
 'Median seq': 1371134.5,
 'N 50': 2695903.0,
 'Min seq': 46366.0,
 'Max seq': 2695903.0}

We can see we have a perfect genome assembly, with one chromosome (the larger sequence, 2.69 mbp) and a plasmid (46 kbp).

### Gene calling, gene sets and prot sets
The next thing we might want to with an assembly is to predict coding sequences (CDS) so we can have a file with genes (which we will call `geneset`) and a file with proteins, which we will call `protset`. For gene calling, we have a wrapper for [Prodigal](https://github.com/hyattpd/Prodigal), a popular software for that:

In [5]:
g.run_prodigal()

2019-09-27 06:10:08 - Starting Prodigal. Your input file is /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.fna. Quiet setting is True.
2019-09-27 06:10:15 - Loaded gene set from Prodigal data. It has 2725 genes.
2019-09-27 06:10:15 - Loaded protein set from Prodigal data.
2019-09-27 06:10:15 - Took 0:00:07.257303


These files are stored in our `files` attribute, which we saw previously:

In [6]:
g.files

{'contigs': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.fna',
 'prodigal': {'genes': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_prodigal_genes.fna',
  'proteins': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_prodigal_proteins.faa',
  'cds': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_prodigal_cds.gbk'}}

Now we have two keys in our `g.files` dictionary: the `'contigs'` key, which holds the file we started with, and the `'prodigal'` keys, which holds a dictionary with the CDS files.

In [7]:
g.files["prodigal"]["proteins"]  # The path to our file holding a set of proteins.

'/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_prodigal_proteins.faa'

Our `Genome` object also loads all of these sequences in memory, through the `geneset` and `protset` attributes:

In [8]:
g.geneset["prodigal"].keys()

dict_keys(['records', 'origin'])

Our `"prodigal"` geneset has all the records from our original genome file. The `"records"` key accesses all of the sequence records in the `geneset`, and the `"origin"` key points to the file from which they were generated:

In [9]:
len(g.geneset["prodigal"]["records"]), g.geneset["prodigal"]["origin"]

(2725,
 '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_prodigal_genes.fna')

Now we have all of these 2725 loaded as a dictionary, which each sequence ID being the key.

In [10]:
g.geneset["prodigal"]["records"]["NC_007604.1_1"]

SeqRecord(seq=Seq('ATGCTTTGGCAAGATTGCGATCAAAGGCTCGGGCAGCCTCCCCCCATGAAGTTG...TAG', SingleLetterAlphabet()), id='NC_007604.1_1', name='NC_007604.1_1', description='NC_007604.1_1 # 65 # 1237 # 1 # ID=1_1;partial=00;start_type=ATG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.570', dbxrefs=[])

In [11]:
# The same goes for the protset:
g.protset["prodigal"]["records"]["NC_007604.1_1"]

SeqRecord(seq=Seq('MLWQDCDQRLGQPPPMKLVCRQNELNTSLSLVSRAVPSRPNHPVLANVLLAADA...RS*', SingleLetterAlphabet()), id='NC_007604.1_1', name='NC_007604.1_1', description='NC_007604.1_1 # 65 # 1237 # 1 # ID=1_1;partial=00;start_type=ATG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.570', dbxrefs=[])

### Annotation

Now that we have predicted CDSs in our genome, we can easily annotate these sequences. Abacat comes with two small pre-packaged databases:  
* [Megares](https://megares.meglab.org/) - an antibiotic resistance genes database and
* Phenotyping - which consists of genes involved in metabolic pathways which define phenotypes, and is still experimental.

To annotate our genomes, we will use the `blast_seqs()` method, which can be adapted for either our gene set or our prot set. In this case, because we want a nucleotide to nucleotide alignment, we will use our gene set with blastn:

In [12]:
g.blast_seqs(db="megares")

2019-09-27 06:10:15 - Blasting GCF_000012525.1_ASM1252v1_genomic to /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_megares_blast.xml.
2019-09-27 06:10:19 - Found 1 hits.

2019-09-27 06:10:19 - Wrote 1 annotated sequences to /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_megares.fasta.
2019-09-27 06:10:19 - Took 0:00:03.947968


NC_007604.1_912 CARD|pvgb|CP002695|3866610-3867801|ARO:3001312|elfamycin|Elfamycins|EF-Tu_inhibition|TUFAB|RequiresSNPConfirmation


Our annotation with Megares only found 1 gene! The annotation data is also stored in our genome object, and can be accessing the database key in the `files` attribute, like so: `g.files['megares']`

It produces 3 files:
* xml - the BLASTn result in XML format.
* annotation - the matching hits with corresponding annotation
* hits - only the hit description, without the sequences.

In [13]:
g.files['megares']

{'xml': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_megares_blast.xml',
 'annotation': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_megares.fasta',
 'hits': '/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_megares.hits'}

### Phenotyping and pathways

The same can be done for the phenotyping database. But, because it is a **protein** database, we can change our search strategy to "blastx", which searches nucleotides against protein sequences.

In [14]:
g.blast_seqs(db="phenotyping", blast="blastx")

2019-09-27 06:10:19 - Blasting GCF_000012525.1_ASM1252v1_genomic to /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_phenotyping_blast.xml.
2019-09-27 06:10:44 - Found 40 hits.

2019-09-27 06:10:44 - Wrote 40 annotated sequences to /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic_phenotyping.fasta.
2019-09-27 06:10:44 - Took 0:00:24.720374


NC_007604.1_9 ornithine.argininosuccinate_synthase.1
NC_007604.1_18 ornithine.keto-hydroxyglutarate-aldolase.1
NC_007604.1_32 ornithine.argD_bifunctional_N-succinyldiaminopimelate-amino.1
NC_007604.1_69 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.1
NC_007604.1_118 mannitol.scrK_Fructokinase.6
NC_007604.1_141 voges.Acetolactate_synthase_small_subunit.2
NC_007604.1_247 sucrose.maltodextrin_phosphorylase.7
NC_007604.1_252 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.1
NC_007604.1_324 galactose.UDP-glucose_4-epimerase.1
NC_007604.1_496 ornithine.NAD-dependent_aldehyde_dehydrogenase.1
NC_007604.1_506 sucrose.glucose-1-phosphate_adenylyltransferase2.3
NC_007604.1_614 sucrose.glucose-1-phosphate_adenylyltransferase.6
NC_007604.1_656 ornithine.argD_bifunctional_N-succinyldiaminopimelate-amino.1
NC_007604.1_694 sorbitol.Sorbitol-6-phosphate_2-dehydrogenase.2
NC_007604.1_839 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.2
NC_007604.1_971 ornithine.argD_bifunct

We can see that the phenotyping database provided many more hits. Abacat provides a custom parser for the phenotyping database, associating each hit with a pathway. We can load this information (after the BLASTx search) using the `run_pathways()` method, and the information will be stored in the `g.pathways` attribute.

In [15]:
g.run_pathways()
g.pathways

2019-09-27 06:10:44 - Found 6 genes for L-arabinose.
2019-09-27 06:10:44 - Found 9 genes for Sucrose_utilization.
2019-09-27 06:10:44 - Found 17 genes for Ornithine_decarboxylase.
2019-09-27 06:10:44 - Found 2 genes for Vogues_proskauer.
2019-09-27 06:10:44 - Found 1 genes for D-galactose_fermetation.
2019-09-27 06:10:44 - Found 0 genes for Celobiose_fermentation.
2019-09-27 06:10:44 - Found 1 genes for D-mannitol.
2019-09-27 06:10:44 - Found 0 genes for Arginine_dihydrolase.
2019-09-27 06:10:44 - Found 0 genes for Thehalose_fermentation.
2019-09-27 06:10:44 - Found 1 genes for D-sorbitol.
2019-09-27 06:10:44 - Found 0 genes for Indole_production.
2019-09-27 06:10:44 - Found 2 genes for M-inositol.
2019-09-27 06:10:44 - Found 1 genes for D-mannose.


{'L-arabinose': ['NC_007604.1_69 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.1',
  'NC_007604.1_252 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.1',
  'NC_007604.1_839 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.2',
  'NC_007604.1_976 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.2',
  'NC_007604.1_1455 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.2',
  'NC_007604.1_2549 arabinose.L-arabinose_transport_ATP-binding_protein_AraG.2'],
 'Sucrose_utilization': ['NC_007604.1_247 sucrose.maltodextrin_phosphorylase.7',
  'NC_007604.1_506 sucrose.glucose-1-phosphate_adenylyltransferase2.3',
  'NC_007604.1_614 sucrose.glucose-1-phosphate_adenylyltransferase.6',
  'NC_007604.1_1001 sucrose.UDP-glucose_6-dehydrogenase.3',
  'NC_007604.1_1051 sucrose.4-alpha-glucanotransferase.3',
  'NC_007604.1_1117 sucrose.1_4-alpha-glucan_branching_enzyme.1',
  'NC_007604.1_2019 sucrose.glucose-1-phosphate_adenylyltransferase2.6',
  'NC_007604.1_2077 s

### Exporting and importing as JSON

Well, now that we made several operations on our original genome file, we want to export our information and be able to retrieve it without running everything again. For that, we provide a JSON parser which exports our `Genome` object to a .json file, which can be easily imported again. It is also useful for communicating with APIs.

In [16]:
g.to_json()

2019-09-27 06:10:44 - Wrote json file of GCF_000012525.1_ASM1252v1_genomic to /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.json.


Let's delete our object and import it again to see what happens. To import it, we will need the `from_json()` function:

In [17]:
from abacat import from_json

In [18]:
del g
g = from_json("/Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.json")

2019-09-27 06:10:44 - Loading genome from /Users/viniWS/Bio/abacat/abacat/data/genomes/GCF_000012525.1_ASM1252v1_genomic.json.
2019-09-27 06:10:44 - Loaded files to GCF_000012525.1_ASM1252v1_genomic.
2019-09-27 06:10:44 - Loaded seqstats to GCF_000012525.1_ASM1252v1_genomic.
2019-09-27 06:10:44 - Loaded pathways to GCF_000012525.1_ASM1252v1_genomic.
2019-09-27 06:10:44 - Loaded gene set from Prodigal data. It has 2725 genes.
2019-09-27 06:10:44 - Loaded protein set from Prodigal data.


All of our information is loaded again!

This wraps up our tutorial. Let's review our steps.

1. Loading the contigs .fasta file with the `Genome` class.
2. Predicting CDSs using Prodigal.
3. Annotating with both the MEGARes and the Phenotyping database.
4. Exporting and importing back again with the `to_json()` and `from_json()` methods.